In [ ]:
import os
import yaml


import numpy as np

import tensorflow as tf
from tensorflow.keras import backend as K
from PIL import Image

LABELS_PATH = "labels.yaml"

In [ ]:
model = tf.keras.models.load_model("output")

In [ ]:
model.summary()

In [ ]:
with open(LABELS_PATH) as f:
    label_map = yaml.load(f, Loader=yaml.SafeLoader)["labels"]
reverse_label_map = {v: k for k, v in label_map.items()}
repr(reverse_label_map)

discard_id = max(reverse_label_map.keys())+1
label_map['discard'] = discard_id
reverse_label_map[discard_id] = 'discard'

image_files = []
GOOD_IMAGES_DIR = "images/esp32/good"
for dir, _, files in os.walk(GOOD_IMAGES_DIR):
    for file in files:
        image_files.append((os.path.basename(dir), os.path.join(dir, file)))

DISCARD_IMAGES_DIR = "images/esp32/discard/"
for dir, _, files in os.walk(DISCARD_IMAGES_DIR):
    for file in files:
        image_files.append(('discard', os.path.join(dir, file)))

print(len(image_files))
print(image_files[0])

In [ ]:
in_layer = model.get_layer('conv2d').input
out_layer = model.get_layer('conv2d_1').output
func = K.function([in_layer], [out_layer])

In [ ]:
img_arrays = []
for img_desc in image_files:
    img = Image.open(img_desc[1])
    img_array = np.asarray(img, dtype=np.uint8).reshape(29, 40, 1)
    formatted_img_array = np.expand_dims(img_array, axis=0)
    img_arrays.append(formatted_img_array)

img_arrays = np.squeeze(np.array(img_arrays), axis=1)
vectors = func(img_arrays)[0]

vectors.shape

In [ ]:
ref = []
i = 350
display(Image.fromarray(np.squeeze(img_arrays[i], axis=2)))
for j in range(vectors.shape[1]):
    display(Image.fromarray((vectors[i][j] * 255).astype(np.uint8)).resize((vectors.shape[3]*10, vectors.shape[2]*10)))
    ref.append((vectors[i][j] * 255).astype(np.uint8))

# i = 1109
# display(Image.fromarray(np.squeeze(img_arrays[i], axis=2)))
# for j in range(vectors.shape[1]):
#     display(Image.fromarray((vectors[i][j] * 255).astype(np.uint8)).resize((vectors.shape[3]*10, vectors.shape[2]*10)))

i = 1120
display(Image.fromarray(np.squeeze(img_arrays[i], axis=2)))
for j in range(vectors.shape[1]):
    display(Image.fromarray((vectors[i][j] * 255).astype(np.uint8)).resize((vectors.shape[3]*10, vectors.shape[2]*10)))

for j in range(vectors.shape[1]):
    b = ref[j] & (vectors[i][j] * 255).astype(np.uint8)
    display(Image.fromarray(b).resize((vectors.shape[3]*10, vectors.shape[2]*10)))


print(reverse_label_map[np.argmax(model.predict(img_arrays[np.newaxis,1120], verbose=False))])
print(np.amax(model.predict(img_arrays[np.newaxis,1120], verbose=False)))